In [2]:
open('cafe.txt', 'w', encoding='utf8').write('café')


4

In [3]:
open('cafe.txt').read()

'café'

In [4]:
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [5]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301' # U+0301 은 `COMBINING ACUTE ACCENT`
s1, s2
len(normalize('NFC',s1)), len (normalize('NFC',s2))

(4, 4)

In [6]:

len(normalize('NFD', s1)), len(normalize('NFD',s2))


(5, 5)

In [7]:
normalize('NFC',s1), normalize('NFC',s2)

('café', 'café')

In [8]:
normalize('NFD',s1), normalize('NFD', s2)


('café', 'café')

In [9]:
normalize('NFC',s1)==normalize('NFC',s2)


True

In [11]:
normalize('NFD',s1)==normalize('NFD',s2)

True

In [12]:
from unicodedata import normalize, name
ohm = '\u2126'
name (ohm)

'OHM SIGN'

In [13]:
ohm_c = normalize('NFC', ohm)
name (ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [14]:
ohm, ohm_c, ohm == ohm_c

('Ω', 'Ω', False)

In [15]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

In [16]:
eszett = 'ß'
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [17]:
eszett_cf = eszett.casefold()
eszett_cf2 = eszett.lower()
eszett, eszett_cf, eszett_cf2

('ß', 'ss', 'ß')

In [18]:
s1 = 'café'
s2 = 'cafe\u0301' # U+0301 COMBINING ACUTE ACCENT
s1 == s2

False

In [20]:
from unicodedata import normalize
def nfc_equal(str1, str2):
  return normalize('NFC', str1) == normalize('NFC', str2)
def fold_equal(str1, str2):
  return (normalize( 'NFC', str1).casefold() ==
    normalize('NFC', str2).casefold())
nfc_equal(s1, s2)


True

In [21]:
nfc_equal('A', 'a')

False

In [23]:
s3 = 'Straße'
s4= 'strasse'
s3 == s4


False

In [25]:
nfc_equal(s3, s4)


False

In [26]:
fold_equal(s3, s4)


True

In [27]:
fold_equal(s1, s2)


True

In [28]:

fold_equal('A', 'a')

True

In [35]:
import unicodedata
import string
def shave_marks (txt):
  norm_txt = unicodedata.normalize('NFD', txt)
  shaved = ''.join(c for c in norm_txt if not unicodedata.combining (c))
  return unicodedata.normalize('NFC', shaved)


In [36]:
order = '"Herr Voẞ: cup of Etker" caffè latte bowl of açaí."'
shave_marks (order)


'"Herr Voẞ: cup of Etker" caffe latte bowl of acai."'

In [37]:
Greek = 'Zéqupoç, Zéfiro'
shave_marks (Greek)

'Zequpoc, Zefiro'

In [38]:

shave_marks ('Straße')


'Straße'

In [39]:
shave_marks ('café')

'cafe'

In [40]:

def shave_marks_latin(txt):
  """Remove all diacritic marks from Latin base characters"""
  norm_txt = unicodedata.normalize('NFD', txt)
  latin_base= False
  keepers = []
  for c in norm_txt:
    if unicodedata.combining (c) and latin_base: continue # ignore diacritic on Latin base char
    keepers.append(c)
  # if it isn't combining char, it's a new base char
  if not unicodedata.combining (c):
    latin_base = c in string.ascii_letters
  shaved = ''.join(keepers)
  return unicodedata.normalize('NFC', shaved)


In [42]:
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)

def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))
    no_marks = no_marks.replace('ß', 'ss')
    return unicodedata.normalize('NFKC', no_marks)

In [43]:

order = '"Herr Voß: cup of Etker" caffè latte bowl of açaí."'
dewinize(order)


'"Herr Voß: cup of Etker" caffè latte bowl of açaí."'

In [44]:
asciize(order)

'"Herr Voss: cup of Etker" caffè latte bowl of açaí."'

In [45]:

fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

In [46]:
!pip install pyuca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.5 MB/s eta 0:00:00


In [47]:
import pyuca
coll = pyuca. Collator()
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [50]:

import unicodedata
import re
re_digit = re.compile(r'\d')
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
  print('U+%04x' % ord (char),
    char.center(6),
    're_dig' if re_digit.match(char) else '-', 'isdig' if char.isdigit() else '-',
    'isnum' if char.isnumeric() else '-', format(unicodedata.numeric(char), '5.2f'), unicodedata.name (char),
    sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


In [65]:
import re
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')
text_str = 'Ramanujan saw \\u0be7\\u0ed\\u0be8\\u0bef as 1729 = 1^3 + 12^3 = 9^3 + 10^3.'
text_bytes = text_str.encode('utf-8')
print('Text', repr(text_str), sep='\n')
print('Numbers')
print('str', re_numbers_str.findall(text_str))
print(' bytes:', re_numbers_bytes.findall (text_bytes))
print('Words')
print('str', re_words_str.findall(text_str))
print(' bytes:', re_words_bytes.findall(text_bytes))

Text
'Ramanujan saw \\u0be7\\u0ed\\u0be8\\u0bef as 1729 = 1^3 + 12^3 = 9^3 + 10^3.'
Numbers
str ['0', '7', '0', '0', '8', '0', '1729', '1', '3', '12', '3', '9', '3', '10', '3']
 bytes: [b'0', b'7', b'0', b'0', b'8', b'0', b'1729', b'1', b'3', b'12', b'3', b'9', b'3', b'10', b'3']
Words
str ['Ramanujan', 'saw', 'u0be7', 'u0ed', 'u0be8', 'u0bef', 'as', '1729', '1', '3', '12', '3', '9', '3', '10', '3']
 bytes: [b'Ramanujan', b'saw', b'u0be7', b'u0ed', b'u0be8', b'u0bef', b'as', b'1729', b'1', b'3', b'12', b'3', b'9', b'3', b'10', b'3']


In [55]:
import os

os.listdir('.')

['.config', 'dummy', 'cafe.txt', 'sample_data']

In [56]:
os.listdir(b'.')

[b'.config', b'dummy', b'cafe.txt', b'sample_data']

In [60]:
os.listdir('./fluent_python')

['.ipynb_checkpoints', 'digits-of-(pi).txt']

In [61]:
os.listdir(b'./fluent_python')


[b'.ipynb_checkpoints', b'digits-of-(pi).txt']

In [62]:
pi_name_bytes = os.listdir (b'.') [1]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
pi_name_str

'.ipynb_checkpoints'